# Day 18: Snailfish

You descend into the ocean trench and encounter some snailfish. They say they saw the sleigh keys! They'll even tell you which direction the keys went if you help one of the smaller snailfish with his math homework.

Snailfish numbers aren't like regular numbers. Instead, every snailfish number is a pair - an ordered list of two elements. Each element of the pair can be either a regular number or another pair.

Pairs are written as [x,y], where x and y are the elements within the pair.

This snailfish homework is about addition. To add two snailfish numbers, form a pair from the left and right parameters of the addition operator.

There's only one problem: snailfish numbers must always be reduced, and the process of adding two snailfish numbers can result in snailfish numbers that need to be reduced.

To reduce a snailfish number, you must repeatedly do the first action in this list that applies to the snailfish number:

- If any pair is nested inside four pairs, the leftmost such pair explodes.
- If any regular number is 10 or greater, the leftmost such regular number splits.

Once no action in the above list applies, the snailfish number is reduced.

During reduction, at most one action applies, after which the process returns to the top of the list of actions. For example, if split produces a pair that meets the explode criteria, that pair explodes before other splits occur.

To explode a pair, the pair's left value is added to the first regular number to the left of the exploding pair (if any), and the pair's right value is added to the first regular number to the right of the exploding pair (if any). Exploding pairs will always consist of two regular numbers. Then, the entire exploding pair is replaced with the regular number 0.

To split a regular number, replace it with a pair; the left element of the pair should be the regular number divided by two and rounded down, while the right element of the pair should be the regular number divided by two and rounded up.

To check whether it's the right answer, the snailfish teacher only checks the magnitude of the final sum. The magnitude of a pair is 3 times the magnitude of its left element plus 2 times the magnitude of its right element. The magnitude of a regular number is just that number.

## Parse and Render

In [ ]:
record struct Value(int depth, int value);

abstract class Number {
    abstract public long Magnitude();
    abstract public Value[] ToValues(int depth);
}

class Regular: Number {
    public int value;

    override public long Magnitude()
        => value;
    override public Value[] ToValues(int depth) 
        => new[]{ new Value(depth, value) };
    override public string ToString()
        => $"{value}";
}

class Pair: Number {
    public Number left;
    public Number right;

    override public long Magnitude()
        => left.Magnitude() * 3 + right.Magnitude() * 2;
    override public Value[] ToValues(int depth) 
        => left.ToValues(depth+1).Concat(right.ToValues(depth+1)).ToArray();
    override public string ToString()
        => $"[{left},{right}]";
}

(Number,string) Parse(string input) {
    var first = input[0];
    var remain = input[1..];
    if (first == '[') {
        (var left, remain) = Parse(remain);
        (var right, remain) = Parse(remain[1..]);
        return (new Pair { left = left, right = right}, remain[1..]);
    }
    return (new Regular { value = first - '0'}, remain);
}

(Number, Value[]) FromValuesAt(Value[] number, int depth) {
    var first = number[0];
    if (depth < first.depth) {
        var (left, remain) = FromValuesAt(number, depth + 1);
        if (remain.Length == 0) return (left, remain);
        (var right, remain) = FromValuesAt(remain, depth + 1);
        return (new Pair { left = left, right = right}, remain);
    }
    return (new Regular { value = first.value }, number[1..]);
}

Number FromValues(Value[] number)
    => FromValuesAt(number, 0).Item1;
    
var s = "[[[[1,3],[5,3]],[[1,3],[8,7]]],[[[4,9],[6,9]],[[8,2],[7,3]]]]";
var n = Parse(s).Item1;
var v = n.ToValues(0);

(n.ToString() == s, FromValues(v).ToString() == s)


Item1,Item2
True,True


## Add


In [ ]:
Value Inc(Value v) {
    v.depth++;
    return v;
}

Value[] Add(Value[] l, Value[] r)
    => l.Select(Inc).Concat(r.Select(Inc)).ToArray();

Number Add(Number l, Number r)
    => new Pair { left = l, right = r};


## Reduce

In [ ]:
(bool,Value[]) Explode(Value[] n) {
    var last = n.Length - 1;
    for (var f = 0; f < last; f++) {
        var s = f + 1;
        if (n[f].depth == 5 && n[s].depth == 5) {
            if (f > 0) n[f-1].value += n[f].value;
            n[f] = new Value(4, 0);
            if (s < last) {
                n[s+1].value += n[s].value;
                return (true, n.Take(s).Concat(n.Skip(s+1)).ToArray());
            } else {
                return (true, n[..s]);
            }
        }
    }
    return (false, n);
}

(bool,Value[]) Split(Value[] n) {
    for (var i = 0; i < n.Length; i++) {
        var v = n[i];
        if (v.value > 9) {
            var h = v.value/2;
            var s = new Value(v.depth+1, v.value-h);
            n[i].depth++;
            n[i].value = h;
            return (true, n[..i].Append(n[i]).Append(s).Concat(n[(i+1)..]).ToArray());
        }
    }
    return (false, n);
}

Number Reduce(Number input) {
    var result = input.ToValues(0);
    var reduce = true;
    while (reduce) {
        (reduce, result) = Explode(result);
        if (!reduce)
            (reduce, result) = Split(result);
        // Console.WriteLine(Render(result));
        }
    return FromValues(result);
}

## Input

In [ ]:
using System.IO;
// var input = File.ReadAllLines(@"day-18.sample");
var input = File.ReadAllLines(@"day-18.input");

var nums = input.Select(i => Parse(i).Item1).ToArray();

nums.Length

100

## Part 1

Add up all of the snailfish numbers from the homework assignment in the order they appear. What is the magnitude of the final sum?

In [ ]:
var sum = nums.Aggregate((l, r) => Reduce(Add(l, r)));

sum.Magnitude()

4132

## Part 2

You notice a second question on the back of the homework assignment:

What is the largest magnitude you can get from adding only two of the snailfish numbers?

Note that snailfish addition is not commutative - that is, x + y and y + x can produce different results.

What is the largest magnitude of any sum of two different snailfish numbers from the homework assignment?

In [ ]:
var sums = nums.SelectMany(x => nums, (x, y) => (a: x, b:y))
    .Where(p => p.a != p.b)
    .Select(p => Reduce(Add(p.a, p.b)))
    .ToArray();

sums.Max(v => v.Magnitude())

4685